In [1]:
## imports and fixed parameters

import ipywidgets as widgets
from IPython.display import display, clear_output, IFrame
from dome_connector import Connector
from cmcl_jobsender import JobSender
from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib as mpl


DOME_URL = 'https://cmcl.dome40.io/'  # DOME platform URL
API_KEY = "6f7d3ae990.fb00950be9a1439d80c6baf515e1f112"  # hard-coded for now
IFRAME_URL = 'http://192.168.1.171:4242/visualisation'
CMCL_URL = 'http://192.168.1.171:4242/'

pd.set_option('display.max_rows', None)

def print_with_indent(obj, indent=0):
    if isinstance(obj, dict):
        for key, value in obj.items():
            print('  ' * indent + str(key) + ':')
            print_with_indent(value, indent + 1)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            print('  ' * indent + f"[{index}]:")
            print_with_indent(item, indent + 1)
    else:
        print('  ' * indent + str(obj))

Textarea(value='', description='Text:', placeholder='Type something')

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

In [ ]:
import os
from urllib.parse import parse_qs

# Get the query string from the environment variable
query_string = os.environ.get('QUERY_STRING', '')

# Parse the query string into a dictionary
params = parse_qs(query_string)
    
if 'mmsi' in params:    
    QUERY_MMSI = params['mmsi'][0]
else:
    raise Exception("MMSI parameter must be supplied in the form of ?mmsi=123456789 in the HTTP request.")

In [ ]:
# search DOME for ship location data - backend

# create connectors

ship_connector = Connector(
    DOME_URL, API_KEY, "fb8490c8-a71a-42c4-bd67-6052cf347f2e")


def parse_ship_data(json_ship):
    metadata = json_ship['metadata']
    data = json_ship['data']
    dict_mmsi = {
        "date": [datetime.fromisoformat(data[0][j]["date"]) for j in range(len(data[0]))],
        "lat": [float(data[0][j]["lat"]) for j in range(len(data[0]))],
        "lon": [float(data[0][j]["lon"]) for j in range(len(data[0]))],
        "speed": [float(data[0][j]["speed"]) for j in range(len(data[0]))],
        "course": [float(data[0][j]["course"]) for j in range(len(data[0]))]
    }
    return dict_mmsi

def get_ship(search_string):
    search_string = search_string or "AIS"
    dict_ship = parse_ship_data(ship_connector.get_data(search_string))
    return pd.DataFrame(dict_ship)

def get_scope(df_ship):
    
    # Calculate MBR
    min_lat, max_lat = min(df_ship['lat']), max(df_ship['lat'])
    min_lon, max_lon = min(df_ship['lon']), max(df_ship['lon'])

    # Calculate Centroid
    centroid_lat = (min_lat + max_lat) / 2
    centroid_lon = (min_lon + max_lon) / 2
    
    return {"LAT": centroid_lat, "LON": centroid_lon,
            "dLAT": max((max_lat - min_lat) * 0.5 * (1 + 0.02), 0.02),
            "dLON": max((max_lon - min_lon) * 0.5 * (1 + 0.02), 0.02)}
    
    

def plot_ship(df_ship,scope):
    # Sample data: list of (latitude, longitude) tuples
    trajectory = list(zip(df_ship['lat'],df_ship['lon']))
    
    # Calculate MBR
    min_lon = scope['LON']-scope['dLON']
    max_lon = scope['LON']+scope['dLON']
    min_lat = scope['LAT']-scope['dLAT']
    max_lat = scope['LAT']+scope['dLAT']

    # Calculate Centroid
    centroid_lat = (min_lat + max_lat) / 2
    centroid_lon = (min_lon + max_lon) / 2
    centroid = (centroid_lat, centroid_lon)

    # MBR corners
    dlat = (max_lat - min_lat) * 0.02
    dlon = (max_lon - min_lon) * 0.02
    mbr_corners = [(min_lat, min_lon),
                   (min_lat, max_lon),
                   (max_lat, max_lon),
                   (max_lat, min_lon),
                   (min_lat, min_lon)]

    # Create a map centered around the first point
    m = folium.Map(location=centroid)
    
    # Add MBR to the map
    # Add filled MBR to the map
    folium.Polygon(locations=mbr_corners,color="green",fill=True,fill_color="green",fill_opacity=0.3,weight=0).add_to(m)

    # Add points to the map
    for i in range(len(trajectory)):
        point = trajectory[i]
        folium.Marker(location=point,popup=folium.Popup(str(df_ship['date'][i]))).add_to(m)
        
    colormap = plt.get_cmap('autumn')
    colors = [colormap(i / len(trajectory)) for i in range(len(trajectory))]

    # Add segments to the map with different colors
    for i in range(len(trajectory) - 1):
        folium.PolyLine(locations=[trajectory[i], trajectory[i + 1]],
                        color=mpl.colors.to_hex(colors[i]),  # Convert RGBA to hex
                        weight=2.5,
                        opacity=1
                        ).add_to(m)
    
    # Fit bounds to ensure everything is visible
    m.fit_bounds([[min_lat-dlat, min_lon-dlon], [max_lat+dlat, max_lon+dlon]])
    
    return m

In [ ]:
# search DOME for ship data - frontend

output_dome_ship = widgets.Output(layout=widgets.Layout())

df_ship = get_ship(QUERY_MMSI)
scope = get_scope(df_ship)

with output_dome_ship:
    output_dome_ship.clear_output()
    display(plot_ship(df_ship,scope))

In [ ]:
# submit job to CMCL server

job_sender = JobSender(CMCL_URL)

# Create a text area widget
text_area_job_label = widgets.Text(
    placeholder="""This will be MMSI:123456789 if not specified.""",
    description="""Name of simulation.""",
    layout=widgets.Layout(flex_flow='row',width='50%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_job_step = widgets.Text(
    placeholder="""All available timestep will be simulated if not specified.""",
    description="""Number of timesteps.""",
    layout=widgets.Layout(flex_flow='row',width='50%'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget
button_job = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Submit',
    icon='check'
)

# Define a function to handle the button click event
def click_button_job(b):
    
    b.button_style = 'info'  # Change the button style
    b.description = 'Clicked!'  # Change the button description

    with output_job:
        output_job.clear_output()
        
        if len(text_area_job_label.value)>0:
            label = text_area_job_label.value
        else:
            label = f"MMSI:{QUERY_MMSI}"
            
        try:
            num_step = int(text_area_job_step.value)
        except:
            num_step = 0
        
        print('Sending ship data...')
        response = job_sender.add_ship_data(QUERY_MMSI,df_ship)
        if response.status_code == 200:
            print('Ship data successfully sent.')
            df_ship['dates'] = pd.to_datetime(df_ship['date'])
            list_timestep = [x.to_pydatetime() for x in df_ship['dates']]
            try:
                if num_step>0:
                    list_timestep = list_timestep[0:num_step]
            except:
                pass
            print(f"Running {len(list_timestep)} timesteps.")
            print_with_indent(job_sender.run_simulation_without_ship(label,scope,list_timestep))

output_job = widgets.Output(layout=widgets.Layout(max_height='100px'))

# Set the event handler for the button click event
button_job.on_click(click_button_job)

In [ ]:
# final-layout

label_dome_ship = widgets.HTML(
value=f'<p style="font-size:24px; font-weight:bold;text-align:center">Location data of ship MMSI:{QUERY_MMSI}</p>'
)

vbox_dome_ship = widgets.VBox(
    #[label_dome_ship, button_dome_ship, output_dome_ship],
    [label_dome_ship, output_dome_ship],
    layout=widgets.Layout(width="50%",align_items='center'))

########################

label_job = widgets.HTML(
value='<p style="font-size:24px; font-weight:bold;text-align:center">Start new simulation</p>'
)

vbox_job = widgets.VBox(
    [label_job, text_area_job_label, text_area_job_step,
        button_job, output_job],
    layout=widgets.Layout(width='80%', align_items='center'))

table = widgets.VBox([vbox_dome_ship, vbox_job],
                     layout=widgets.Layout(align_items='center'), width='100%')

display(table)